In [2]:
#notebook based on zflemings:https://nbviewer.jupyter.org/github/zflamig/dask-era5/blob/main/notebook/era5_fargate_dask.ipynb
import xarray as xr
import fsspec
import dask
import s3fs
import numpy as np
xr.set_options(display_style="html")  #display dataset nicely 

NameError: name 's3fs' is not defined

In [21]:
#ds = xr.open_zarr('https://era5-pds.s3.us-east-1.amazonaws.com/zarr/1983/05/data/northward_wind_at_10_metres.zarr')#,consolidated=True)
#fname = ' http://era5-pds.s3.amazonaws.com/zarr/2008/01/data/air_temperature_at_2_metres.zarr'
fname = 'https://era5-pds.s3.us-east-1.amazonaws.com/zarr/1983/05/data/air_pressure_at_mean_sea_level.zarr'
#ds = xr.open_zarr(fname)#,consolidated=True)
#ds


fs = s3fs.S3FileSystem(anon=True)
datestring = 'era5-pds/zarr/1983/05/data/air_pressure_at_mean_sea_level.zarr/'
mslp_zarr = xr.open_zarr(s3fs.S3Map(datestring, s3=fs))
#mslp_zarr

In [ ]:
#url = 's3://era5-pds/2010/01/data/air_temperature_at_2_metres.nc'
url = 's3://era5-pds/2010/01/data/precipitation_amount_1hour_Accumulation.nc'
ncfile = fsspec.open(url)
ds = xr.open_dataset(ncfile.open())
ds

In [2]:
years,months=[],[]
[years.append(str(y)) for y in np.arange(1979,2020)]
[months.append(str(m).zfill(2)) for m in np.arange(1,13)]
#years = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
#months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
file_pattern = 's3://era5-pds/{year}/{month}/data/precipitation_amount_1hour_Accumulation.nc' #air_temperature_at_2_metres.nc'

@dask.delayed
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
    return fs.open(path)

files_mapper = [s3open(file_pattern.format(year=year,month=month)) for year in years for month in months]

In [3]:
%%time

ds = xr.open_mfdataset(files_mapper, engine='h5netcdf', chunks={'lon':200,'lat':200,'time1':720}, concat_dim='time1', combine='nested', coords='minimal', compat='override', parallel=True)

ds

CPU times: user 49 s, sys: 1min 4s, total: 1min 53s
Wall time: 8min 9s


<xarray.Dataset>
Dimensions:                                  (lat: 721, lon: 1440, nv: 2, time1: 359393)
Coordinates:
  * lon                                      (lon) float32 0.0 0.25 ... 359.75
  * lat                                      (lat) float32 90.0 89.75 ... -90.0
  * time1                                    (time1) datetime64[ns] 1979-01-0...
Dimensions without coordinates: nv
Data variables:
    time1_bounds                             (time1, nv) datetime64[ns] dask.array<chunksize=(720, 2), meta=np.ndarray>
    precipitation_amount_1hour_Accumulation  (time1, lat, lon) float32 dask.array<chunksize=(720, 200, 200), meta=np.ndarray>
Attributes:
    source:       Reanalysis
    institution:  ECMWF
    title:        ERA5 forecasts

In [ ]:
#%%time
#ds = xr.open_mfdataset(files_mapper, engine='h5netcdf', chunks={'lon':200,'lat':200,'time0':720}, concat_dim='time0', combine='nested', coords='minimal', compat='override', parallel=True)
#ds

In [4]:
ds_daily = ds.resample(time1='1D').sum('time1',keep_attrs=True,skipna=False)
ds_daily    

,Array,Chunk
Bytes,62.19 GB,160.00 kB
Shape,"(14975, 721, 1440)","(1, 200, 200)"
Count,2446412 Tasks,479200 Chunks
Type,float32,numpy.ndarray


In [ ]:
ds_daily.precipitation_amount_1hour_Accumulation[-1,:,:].plot(vmin=0,vmax=0.001)

In [5]:
ds_cali = ds_daily.sel(lat=slice(42,32),lon=slice(236,246)).sum({'lat','lon'}).chunk({'time1':len(ds_daily.time1)})
ds_cali

<xarray.Dataset>
Dimensions:                                  (time1: 14975)
Coordinates:
  * time1                                    (time1) datetime64[ns] 1979-01-0...
Data variables:
    precipitation_amount_1hour_Accumulation  (time1) float32 dask.array<chunksize=(14975,), meta=np.ndarray>

In [6]:
ds_cali.to_zarr('./../../data/cali_box_24hr_precip_zarr2')

In [ ]:
ds_cali.precipitation_amount_1hour_Accumulation.plot()

In [ ]:
import numpy as np
ds_cali.sel(time1=slice(np.datetime64('2019-01-01'),np.datetime64('2019-01-31'))).precipitation_amount_1hour_Accumulation.plot()

In [ ]:
ds_cali.sel(time1=slice(np.datetime64('2019-01-01'),np.datetime64('2019-01-31'))).precipitation_amount_1hour_Accumulation.plot()

In [ ]:
storm_list = ds_cali.time1.where(ds_cali.precipitation_amount_1hour_Accumulation>20,drop=True)

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
cluster = gateway.new_cluster(worker_memory=8)
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster